In [27]:
import numpy as np
import pandas as pd

from utils import *
from viz import *

import plotly.express as px
import plotly.graph_objects as go


In [28]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
# px.bar_polar?

In [30]:
tasks = read_tasks('data/example1.txt')
traces = read_traces('data/trace2.txt')

In [31]:
df_traces = process_traces(traces)
df_traces.head()

,id,dt,end,start,task
0,3,1,1,0,3
1,2,1,2,1,2
2,1,1,3,2,1
3,0,2,5,3,3
4,2,1,6,5,2


In [32]:
plot_polar(tasks, df_traces, 600)

In [33]:
plot_gantt(tasks, df_traces)